In [ ]:
def powerset(s):
    x = len(s)
    liste = []
    for i in range(1 << x):
        liste.append(([s[j] for j in range(x) if (i & (1 << j))]))
    return liste

In [ ]:
def std_dev(data):
    # Get number of observations
    n = len(data)
    # Calculate mean
    mean = sum(data) / n
    # Calculate deviations from the mean
    deviations = sum([(x - mean)**2 for x in data])
    # Calculate Variance & Standard Deviation
    variance = deviations / (n - 1)
    s = variance**(1/2)
    return s


# Sharpe Ratio From Scratch
def sharpe_ratio(data, risk_free_rate=0.0):
    # Calculate Average Daily Return
    mean_daily_return = sum(data) / len(data)
    # Calculate Standard Deviation
    s = std_dev(data)
    # Calculate Daily Sharpe Ratio
    daily_sharpe_ratio = (mean_daily_return - risk_free_rate) / s
    # Annualize Daily Sharpe Ratio
    sharpe_ratio = 252**(1/2) * daily_sharpe_ratio

    return sharpe_ratio


In [ ]:
powerset([1,2,3])

In [ ]:
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
options = pd.read_csv("/Users/Bjorn/Desktop/Master/Schoder Seminar/Paper/Datenanalyse/Data/Options/Collection/options_reduced.csv", parse_dates =["Expiration", "DataDate"])
options.drop(options[options["Type"] == "put"].index, inplace = True)
optionsdrop = options.drop(options[options["Last"] == 0].index)
optionsdrop.head()


In [ ]:
optionsdrop[optionsdrop["OptionRoot"] == "MCO180817C00105000"]

In [ ]:
mco = pd.read_csv("Datenanalyse/Data/MCO.csv", index_col = "Date", parse_dates = True)
nee = pd.read_csv("Datenanalyse/Data/NEE.csv", index_col = "Date", parse_dates = True)
rf = pd.read_csv("Datenanalyse/Data/RF.csv", index_col = "Date", parse_dates = True)
tfc = pd.read_csv("Datenanalyse/Data/TFC.csv", index_col = "Date", parse_dates = True)
txn = pd.read_csv("Datenanalyse/Data/TXN.csv", index_col = "Date", parse_dates = True)

mco.reset_index(inplace = True)
mco.drop(columns = ["Unnamed: 0"], inplace = True)
mco["UnderlyingSymbol"] = "MCO"

nee.reset_index(inplace = True)
nee.drop(columns = ["Unnamed: 0"], inplace = True)
nee["UnderlyingSymbol"] = "NEE"

rf.reset_index(inplace = True)
rf.drop(columns = ["Unnamed: 0"], inplace = True)
rf["UnderlyingSymbol"] = "RF"

tfc.reset_index(inplace = True)
tfc.drop(columns = ["Unnamed: 0"], inplace = True)
tfc["UnderlyingSymbol"] = "TFC"

txn.reset_index(inplace = True)
txn.drop(columns = ["Unnamed: 0"], inplace = True)
txn["UnderlyingSymbol"] = "TXN"

lists = [mco, nee, rf, tfc, txn]
symbols = pd.concat(lists)
symbols.sort_values("Date", inplace = True)
#symbols = symbols[(symbols["Date"]>= "2018-05-05") & (symbols["Date"]>= "2018-07-05")]
symbols


In [ ]:
mco.head(10000000)

In [ ]:
def generateRandomTableold(min_date, max_date, symbols, rowCount):

    df = pd.DataFrame(columns =["startDate","endDate","symbols","lossProfitStock", "lossProfitOption", "winner"])

    #generate randomized start and enddates
    time_between_dates = max_date - min_date
    days_between_dates = time_between_dates.days

    for row in range(rowCount):

        random_number_of_days1 = randrange(days_between_dates)
        random_date1 = min_date + timedelta(days=random_number_of_days1)
        random_number_of_days2 = randrange(days_between_dates)
        random_date2 = min_date + timedelta(days=random_number_of_days2)

        #sort dates if needed
        if random_date1 > random_date2:
            temp = random_date1
            random_date1 = random_date2
            random_date2 = temp


        symbolAdd = random.randint(1,6)
        #sample to ensure uniquness of indizes
        symbolAddArray = sample(range(0,5), symbolAdd)
        symbolAddArray.sort()
        #transfer indices to symbols
        symbolRandomList = [ symbols[i] for i in symbolAddArray]


        df = df.append({"startDate": random_date1, "endDate": random_date2, "exitStrategy": "stopFive", "symbols": symbolRandomList}, ignore_index = True)

    return df

In [ ]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from numpy import random
from random import randrange
from random import sample
import warnings
from pandas.core.common import SettingWithCopyWarning
from functools import reduce

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
def generateRandomTable():

    df = pd.DataFrame(columns =["startDate","endDate","symbols","lossProfitStock", "lossProfitOption", "winner", "Sharpe_Ratio Stock", "Sharpe_Ratio Option"])

    #generate randomized start and enddates
    date1 = date(2018, 5, 5)
    date2 = date(2018, 7, 5)

    power = powerset(["TXN", "MCO", "RF", "NEE"])

    for row in range(1, len(power)):
        #symbolAdd = random.randint(1,5)
        #sample to ensure uniquness of indizes

        symbolAddArray = power[row]
        #symbolAddArray.sort()
        #transfer indices to symbols
        #symbolRandomList = [ symbols[i] for i in symbolAddArray]


        df = df.append({"startDate": date1, "endDate": date2, "exitStrategy": "", "symbols": symbolAddArray}, ignore_index = True)

    return df

In [ ]:
#define initial values
rowCount = 10
#stocks = ["TXN", "MCO", "RF", "NEE"]

#create random table
df1 = generateRandomTable()
df1
#create csv
#df1.to_csv("simulation_data.csv", index=False, encoding='utf-8-sig')

In [ ]:
def remove_from_array(base_array, num):
    for index in range(len(base_array)):
        if (base_array[index]["ID"].values == num):
            base_array.pop(index)
            break

In [ ]:
def stock_vs_options(start_date, end_date, exitStrat, work, symbols, options, shortEnable, riskfreerate):
    starting_capital = 1000000
    stockInvestor = investor(1, starting_capital, symbols, options)
    optionInvestor = investor(2, starting_capital, symbols, options)
    tasks = {}
    for symbol in work:
        #tasks[symbol] = np.random.randint(1,12)
        tasks[symbol] = 100

    #stockInvestor.buy(tasks, exitStrat, start_date, end_date, shortEnable)
    sharpeStock = stockInvestor.stockinvestNew(tasks, exitStrat, start_date, end_date, shortEnable, riskfreerate)
    
    for symbol in work:
        optionInvestor.callinvest(symbol, start_date, end_date, 0.1, 0.15)
        
    #stockProfit = ((stockInvestor.assetValue - starting_capital)/stockInvestor.totalInvestedCapital) * 100
    stockProfit = ((stockInvestor.capital - starting_capital)/stockInvestor.totalInvestedCapital) * 100
    optionProfit = ((optionInvestor.capital - starting_capital)/optionInvestor.totalInvestedCapital) * 100
    
    #print("Stock Investor has created a loss/profit of "+ str(stockProfit))
    #print("Option Investor has created a loss/profit of "+ str(optionProfit))

    return [stockProfit, optionProfit, sharpeStock[0], sharpeStock[1]]


In [ ]:
stock_vs_options("2018-05-05", "2018-05-15", "", ["MCO"], symbols, optionsdrop, True)

In [ ]:
def runSimulation(data, stock, options, shortEnable, riskfreerate):
    for index, row in data.iterrows():
        win = ""
        temp = stock_vs_options(row.startDate, row.endDate, row.exitStrategy, row.symbols, stock, options, shortEnable, riskfreerate)
        data.at[index, "lossProfitStock"] = temp[0]
        data.at[index, "lossProfitOption"] = temp[1]
        data.at[index, "Sharpe_Ratio Stock"] = temp[3]
        data.at[index, "Sharpe_Ratio Option"] = 0
        if (temp[0] > temp[1]): win = "Stock"
        elif (temp[0] < temp[1]): win = "Option"
        elif (temp[0] == temp[1]): win = "En par"
        data.at[index, "winner"] = win
    return data

In [ ]:
res2 = runSimulation(df1, symbols, optionsdrop, True, 0.0)
res2

In [ ]:
res = runSimulation(df1, symbols, optionsdrop, True)
res

In [ ]:
res2 = runSimulation(df1, symbols, optionsdrop, True)
res2

In [ ]:
stock_vs_options("2018-05-09", "2018-07-20", "stopFive", ["MCO"], symbols, optionsdrop)

In [ ]:
res.to_csv("/Users/Bjorn/Desktop/Master/Schoder Seminar/Paper/Zwischenergebnisse Statistische Analyse/hoheWerte.csv")

In [ ]:
class investor:
    def __init__(self, no, capital, stockdata, optionsdata):
        self.no = no
        self.stockdata = stockdata
        self.optionsdata = optionsdata
        self.capital = capital
        self.initial = capital
        self.assetList = {}
        self.shortassetList = {}
        self.symbolCapital = {}
        self.boughtCalls = []
        self.soldCalls = []
        self.assetValue = 0
        self.totalInvestedCapital = 0

    def compete(self, tasklist):
        counter = 1
        workers = {}
        for task in tasklist:
            worker = investor(counter, self.capital, self.stockdata, self.optionsdata)
            worker.buy(task[0], task[1], task[2], task[3])
            workers[counter] = worker.capital - self.capital
            counter += 1
        for w in workers:
            print("Worker with number " + str(w) + " has created a profit of "+ str(workers[w]))

    def getAssets(self, date):
        date = pd.to_datetime(np.datetime64(date))
        checker = False
        while (checker == False):
            if (len(self.stockdata[self.stockdata.Date == date]) > 0):
                break
            else:
                date = date - pd.to_timedelta(1, "D")
        for stock in self.assetList:
            if (self.assetList[stock] > 0):
                self.assetValue += (self.stockdata[(self.stockdata.Date == date)&(self.stockdata.UnderlyingSymbol == stock)].Close.values) * self.assetList[stock]

        self.assetValue = self.assetValue + self.capital



    def stockinvest(self, shareno, symbol, exitS, entryD, exitD, shortEnable):
        entryD = pd.to_datetime(np.datetime64(entryD))
        exitD = pd.to_datetime(np.datetime64(exitD))
        date = entryD
        portfolioValue = []
        initCapital = self.totalInvestedCapital
        currentPortfolioValue = self.capital
        while (date <= exitD):
            seller = False
            temp = self.stockdata[(self.stockdata.Date == date) & (self.stockdata.UnderlyingSymbol == symbol)]
            if ((date == exitD) & (exitS == "sellAll")):
                seller = True
                checker = False
                while (checker == False):
                    if (len(self.stockdata[self.stockdata.Date == date]) > 0):
                        break
                    else:
                        date = date + pd.to_timedelta(1, "D")
                temp_sell = self.stockdata[(self.stockdata.Date == date) & (self.stockdata.UnderlyingSymbol == symbol)]

            if ((temp["UnderlyingSymbol"].values.size > 0) | seller == True):
                if (seller == False):
                    if ((date >= entryD) & (date < exitD) & (temp["UnderlyingSymbol"].values == symbol)):
                        if (((temp["TradingPoints"].values == "1") | (temp["TradingPoints"].values == "3")) & (shortEnable == True)):
                            if (symbol in self.shortassetList):
                                self.capital = self.capital + temp.Close.values*shareno
                                self.shortassetList[symbol] = self.shortassetList[symbol] + shareno
                            else:
                                self.shortassetList[symbol] = 0
                                self.capital = self.capital + temp.Close.values*shareno
                                self.shortassetList[symbol] = self.shortassetList[symbol] + shareno
                        if ((temp["TradingPoints"].values == "0") | (temp["TradingPoints"].values == "2")):
                            self.capital = self.capital - temp.Close.values*shareno
                            self.totalInvestedCapital = self.totalInvestedCapital + temp.Close.values*shareno
                            if (symbol in self.assetList):
                                self.assetList[symbol] = self.assetList[symbol] + shareno
                                #print("Now buying " + str(shareno) + " Shares of Share " + symbol)
                                #self.getAssetList()
                            else:
                                self.assetList[symbol] = 0
                                self.assetList[symbol] = self.assetList[symbol] + shareno
                                #print("Now buying " + str(shareno) + " Shares of Share " + symbol)
                                #self.getAssetList()
                        if (temp["TradingPoints"].values == "4"):
                            if(symbol in self.assetList):
                                if(self.assetList[symbol] != 0):
                                    self.capital = self.capital + temp.Close.values*round(self.assetList[symbol]*0.5)
                                    #print("Now selling " + str(round(self.assetList[symbol]*0.5)) + " Shares of Share " + symbol)
                                    self.assetList[symbol] -= round(self.assetList[symbol] * 0.5)
                               # self.getAssetList()
                            if((symbol in self.shortassetList) & (shortEnable == True)):
                                if(self.shortassetList[symbol] != 0):
                                    self.capital = self.capital - temp.Close.values*round(self.shortassetList[symbol]*0.5)
                                    self.totalInvestedCapital = self.totalInvestedCapital + temp.Close.values*round(self.shortassetList[symbol]*0.5)
                                    #print("Now selling " + str(round(self.assetList[symbol]*0.5)) + " Shares of Share " + symbol)
                                    self.shortassetList[symbol] -= round(self.shortassetList[symbol] * 0.5)
                            # self.getAssetList()
                        if (temp["TradingPoints"].values == "5"):
                            if((symbol in self.assetList) ):
                                if((self.assetList[symbol] != 0)):
                                    self.capital = self.capital + temp.Close.values*self.assetList[symbol]
                                    self.assetList[symbol] = 0
                                    #print("Now sold all Shares of Share " + symbol)
                                   # self.getAssetList()
                                    if (exitS == "stopFive"):
                                        break
                            if((symbol in self.shortassetList)):
                                if((self.shortassetList[symbol] != 0) & (shortEnable == True)):
                                    self.capital = self.capital - temp.Close.values*self.shortassetList[symbol]
                                    self.totalInvestedCapital = self.totalInvestedCapital + temp.Close.values*self.shortassetList[symbol]
                                    self.shortassetList[symbol] = 0
                                    #print("Now sold all Shares of Share " + symbol)
                                    # self.getAssetList()
                                    if (exitS == "stopFive"):
                                        break
                if ((date >= exitD) & (exitS == "sellAll")):
                    print(date)
                    self.capital = self.capital + temp_sell.Close.values*self.assetList[symbol]
                    self.assetList[symbol] = 0
                   # print("Now sold all Shares")
                  #  self.getAssetList()
                    break
            if ((symbol in self.assetList) & (len(temp.Close.values) >0)):
                portfolioValue.append(self.assetList[symbol]*temp.Close.values[0])
            else:
                portfolioValue.append(0)
            #print(portfolioValue)
            date += timedelta(days=1)
            #self.getAssetList()
        #print(np.var(portfolioValue))

        return [portfolioValue, self.totalInvestedCapital - initCapital]
        #print("The profit that was generated by buying and selling stocks of " + str(((self.capital - self.initial)/self.totalInvestedCapital) * 100))

    def stockinvestNew(self, task, exitS, entryD, exitD, shortEnable, riskfreerate):
        entryD = pd.to_datetime(np.datetime64(entryD))
        exitD = pd.to_datetime(np.datetime64(exitD))
        date = entryD
        portfolioValue = []
        #initCapital = self.capital
        currentPortfolioValue = self.capital
        TWRhelp = []
        towinitialinvestedcapital = self.totalInvestedCapital
        capitalstarttimeline = self.capital
        cashFlow = 0
        while (date <= exitD):
            changeMade = False
            for i in task:
                symbol = i
                shareno = task[i]
                seller = False
                temp = self.stockdata[(self.stockdata.Date == date) & (self.stockdata.UnderlyingSymbol == symbol)]
                if ((date == exitD) & (exitS == "sellAll")):
                    seller = True
                    checker = False
                    while (checker == False):
                        if (len(self.stockdata[self.stockdata.Date == date]) > 0):
                            break
                        else:
                            date = date + pd.to_timedelta(1, "D")
                    temp_sell = self.stockdata[(self.stockdata.Date == date) & (self.stockdata.UnderlyingSymbol == symbol)]

                if ((temp["UnderlyingSymbol"].values.size > 0) | seller == True):
                    if (seller == False):
                        if ((date >= entryD) & (date < exitD) & (temp["UnderlyingSymbol"].values == symbol)):
                            if (((temp["TradingPoints"].values == "1") | (temp["TradingPoints"].values == "3")) & (shortEnable == True)):
                                if (symbol in self.shortassetList):
                                    self.capital = self.capital + temp.Close.values*shareno
                                    self.shortassetList[symbol] = self.shortassetList[symbol] + shareno
                                    #cashFlow += temp.Close.values*shareno
                                else:
                                    self.shortassetList[symbol] = 0
                                    self.capital = self.capital + temp.Close.values*shareno
                                    self.shortassetList[symbol] = self.shortassetList[symbol] + shareno
                                   # cashFlow += temp.Close.values*shareno
                            if ((temp["TradingPoints"].values == "0") | (temp["TradingPoints"].values == "2")):
                                self.capital = self.capital - temp.Close.values*shareno
                                self.totalInvestedCapital = self.totalInvestedCapital + temp.Close.values*shareno
                                cashFlow += temp.Close.values*shareno
                                changeMade = True
                                if (symbol in self.assetList):
                                    self.assetList[symbol] = self.assetList[symbol] + shareno
                                    #print("Now buying " + str(shareno) + " Shares of Share " + symbol)
                                    #self.getAssetList()
                                else:
                                    self.assetList[symbol] = 0
                                    self.assetList[symbol] = self.assetList[symbol] + shareno
                                    #print("Now buying " + str(shareno) + " Shares of Share " + symbol)
                                    #self.getAssetList()
                            if (temp["TradingPoints"].values == "4"):
                                if(symbol in self.assetList):
                                    if(self.assetList[symbol] != 0):
                                        self.capital = self.capital + temp.Close.values*round(self.assetList[symbol]*0.5)
                                        cashFlow -= temp.Close.values*round(self.assetList[symbol]*0.5)
                                        #print("Now selling " + str(round(self.assetList[symbol]*0.5)) + " Shares of Share " + symbol)
                                        self.assetList[symbol] -= round(self.assetList[symbol] * 0.5)
                                        changeMade = True
                                # self.getAssetList()
                                if((symbol in self.shortassetList) & (shortEnable == True)):
                                    if(self.shortassetList[symbol] != 0):
                                        self.capital = self.capital - temp.Close.values*round(self.shortassetList[symbol]*0.5)
                                        #cashFlow -= temp.Close.values*round(self.shortassetList[symbol]*0.5)
                                        self.totalInvestedCapital = self.totalInvestedCapital + temp.Close.values*round(self.shortassetList[symbol]*0.5)
                                        #print("Now selling " + str(round(self.assetList[symbol]*0.5)) + " Shares of Share " + symbol)
                                        self.shortassetList[symbol] -= round(self.shortassetList[symbol] * 0.5)
                                # self.getAssetList()
                            if (temp["TradingPoints"].values == "5"):
                                if((symbol in self.assetList) ):
                                    if((self.assetList[symbol] != 0)):
                                        self.capital = self.capital + temp.Close.values*self.assetList[symbol]
                                        cashFlow -= temp.Close.values*self.assetList[symbol]
                                        self.assetList[symbol] = 0
                                        changeMade = True
                                        #print("Now sold all Shares of Share " + symbol)
                                        # self.getAssetList()
                                        if (exitS == "stopFive"):
                                            break
                                if((symbol in self.shortassetList)):
                                    if((self.shortassetList[symbol] != 0) & (shortEnable == True)):
                                        self.capital = self.capital - temp.Close.values*self.shortassetList[symbol]
                                        #cashFlow -= temp.Close.values*self.shortassetList[symbol]
                                        self.totalInvestedCapital = self.totalInvestedCapital + temp.Close.values*self.shortassetList[symbol]
                                        self.shortassetList[symbol] = 0
                                        #print("Now sold all Shares of Share " + symbol)
                                        # self.getAssetList()
                                        if (exitS == "stopFive"):
                                            break
                    if ((date >= exitD) & (exitS == "sellAll")):
                        print(date)
                        self.capital = self.capital + temp_sell.Close.values*self.assetList[symbol]
                        self.assetList[symbol] = 0
                        # print("Now sold all Shares")
                        #  self.getAssetList()
                        break
            if(self.totalInvestedCapital > 0):
                portfolioValue.append(((self.capital- currentPortfolioValue)/self.totalInvestedCapital)[0])
            else:
                portfolioValue.append(0)

            currentPortfolioValue = self.capital

            if (changeMade == True):
                if ((self.totalInvestedCapital - towinitialinvestedcapital) != 0):
                    TWRhelp.append(((self.capital - (capitalstarttimeline + cashFlow))/(cashFlow + capitalstarttimeline))[0])



                #towinitialinvestedcapital = self.totalInvestedCapital
                capitalstarttimeline = self.capital
                cashFlow = 0
                changeMade = False


            date += timedelta(days=1)

        for r in self.assetList:
            if (self.assetList[r] > 0):
                temp = self.stockdata[(self.stockdata.Date == date) & (self.stockdata.UnderlyingSymbol == r)]
                self.capital = self.capital + temp.Close.values*self.assetList[r]
                self.assetList[r] = 0

        for s in self.shortassetList:
            if (self.shortassetList[s] > 0):
                temp = self.stockdata[(self.stockdata.Date == date) & (self.stockdata.UnderlyingSymbol == s)]
                self.capital = self.capital - temp.Close.values*self.assetList[s]
                self.shortassetList[s] = 0

            #print(self.assetList)
            #print(self.shortassetList)

        TWR = reduce(lambda x, y: (1+x)*(1+y), TWRhelp)
        TWR = TWR -1


        #print(TWRhelp)
        #print(TWR)
        return [sharpe_ratio(portfolioValue, riskfreerate), TWR]
        #return [portfolioValue, (self.capital- initCapital)/self.totalInvestedCapital]




    def buy(self, task, exitS, entryD, exitD, shortEnable):
        portfolios = []
        investedPer = []
        investedTotal = []
        invested= []
        for i in task:
            if (i in self.assetList):
                self.assetList[i] += task[i]
                portfolios.append(self.stockinvest( task[i], i, exitS, entryD, exitD, shortEnable)[0])
                invested.append(self.stockinvest( task[i], i, exitS, entryD, exitD, shortEnable)[1])
            else:
                self.assetList[i] = 0
                self.symbolCapital[i] = self.capital
                portfolios.append(self.stockinvest(task[i], i, exitS, entryD, exitD, shortEnable)[0])
                invested.append(self.stockinvest( task[i], i, exitS, entryD, exitD, shortEnable)[1])

        #print(portfolios)
        #print(invested)

        investedTotal = [sum(x) for x in zip(*portfolios)]
        print(investedTotal)

        stdTotal = std_dev(investedTotal)
        #print(stdTotal)

        xTEMP = pd.Series(investedTotal)
        pctChange = xTEMP.pct_change()
        print(pctChange)

        SR = sharpe_ratio(investedTotal, risk_free_rate=0.0)
        #print(SR)

        summe = sum(invested)
        for r in range(len(invested)):
            if (summe != 0):
                investedPer.append(invested[r]/summe)
            else:
                investedPer.append(-1)

        print(invested)
        #print(investedPer)


        #new = [a+b for a, b in zip(l1, l2)]
        #assetResult = [a+b for a,b in zip(portfolios)]
        #assetResult = [sum(x) for x in zip(*portfolios)]
        #print(assetResult)
        #print(portfolios)
        #print(portfolios[0])
        #for s in range(len(assetResult)):
        #    if assetResult[s] == 0:
        #        assetResult[s] = -1
        #for r in range(len(portfolios)):
         #   res = [i / j for i, j in zip(portfolios[r], assetResult)]
         #   sharePerDay.append(res)

        #for v in range(len(portfolios)):
        #    res = [i * j for i, j in zip(portfolios[v], sharePerDay[v])]
        #    weightedResults.append(res)


        #print(sharePerDay)
        #print(weightedResults)


        #self.getAssets(exitD)
        #print("The remaining capital amounts to: " + str(self.capital))
        #print("The total profit that was generated in this round amounts to " +str(self.capital - self.initial))



    def getAssetList(self):
        for i in self.assetList:
            print("The worker owns " + str(self.assetList[i]) + " of Asset " + i)

    def optionsPreprocessing(self):
        self.optionsdata.drop(self.optionsdata[self.optionsdata["Type"] == "put"].index, inplace = True)


    def buyCalls(self, df, stockprice, percentage, date, symbol, future, lower, buyer, startDate):
        future = pd.to_timedelta(future, "D")
        df = df[df["DataDate"] == date]
        df = df[df["UnderlyingSymbol"] == symbol]
        targetDate = date + future
        strikeTarget = stockprice * (1 + percentage)
        df["buyable"] = True
        df["Index"] = np.arange(0,len(df))
        df["targetDatePossible"] = False
        solution = 0
        checker = False
        boundary = 0
        while ((checker == False) & (targetDate >= startDate)):
           # print(targetDate)
            counter = 0
            for j,k in df.iterrows():
                if (k.Expiration == targetDate):
                    df.at[j, "targetDatePossible"] = True
                    k.targetDatePossible = True
                    counter += 1
                    if (counter >= 2):
                        checker = True
            if (checker == False):
                if (lower == False):
                   # print(targetDate)
                    targetDate = targetDate + pd.to_timedelta(1, "D")
                if (lower == True):
                    targetDate = targetDate - pd.to_timedelta(1, "D")
                boundary += 1
                if (boundary >= 600):
                    lower = True
                if (targetDate <= startDate):
                    break;
        #print(targetDate)
        df = df[df["targetDatePossible"] == True]
        if (len(df) > 0):
            df2 = df.sort_values("Strike", inplace = False)
            df2["Index"] = np.arange(0,len(df2))
            #print("The selected Target Date is now: "  + str(targetDate))
            check = False
            for i in df2["Index"]:
                if((df2[df2["Index"] == i].Strike.values > strikeTarget)):
                    if (i == 0):
                        solution = i
                        break;
                    val1 = df2[df2["Index"] == i-1].Strike.values
                    val2 = df2[df2["Index"] == i].Strike.values
                    temp1 = abs(strikeTarget - val1)
                    temp2 = abs(strikeTarget - val2)
                    check = True
                    if ((temp1 < temp2) | (buyer == True)):
                        solution = i-1
                    else:
                        solution = i
                    break;
            if (check == False):
                val1 = df2[df2["Index"] == len(df2)-2].Strike.values
                val2 = df2[df2["Index"] == len(df2)-1].Strike.values
                temp1 = abs(strikeTarget - val1)
                temp2 = abs(strikeTarget - val2)
                if ((temp1 < temp2) | (buyer == True)):
                    solution = len(df2)-2
                else:
                    solution = len(df2)-1

            return df2[df2["Index"] == solution]
        if (len(df) <= 0):
            return df

    def sellCalls(self, df, date, symbol, future, startDate, baughtStrike, lower):
        future = pd.to_timedelta(future, "D")
        df = df[df["DataDate"] == date]
        df = df[df["UnderlyingSymbol"] == symbol]
        targetDate = date + future
        df["buyable"] = True
        df["Index"] = np.arange(0,len(df))
        df["targetDatePossible"] = False
        solution = 0
        checker = False
        boundary = 0
        while ((checker == False) & (targetDate >= startDate)):
            # print(targetDate)
            counter = 0
            for j,k in df.iterrows():
                if (k.Expiration == targetDate):
                    df.at[j, "targetDatePossible"] = True
                    k.targetDatePossible = True
                    counter += 1
                    if (counter >= 2):
                        checker = True
            if (checker == False):
                if (lower == False):
                    # print(targetDate)
                    targetDate = targetDate + pd.to_timedelta(1, "D")
                if (lower == True):
                    targetDate = targetDate - pd.to_timedelta(1, "D")
                boundary += 1
                if (boundary >= 600):
                    lower = True
                if (targetDate <= startDate):
                    break;
        #print(targetDate)
        df = df[df["targetDatePossible"] == True]
        #print(df[df["Strike"] > baughtStrike[0]])
        df = df[df["Strike"] > baughtStrike[0]]
        if (len(df) > 0):
            df2 = df.sort_values("Last", ascending = False, inplace = False)
            temporary = df2.iloc[0]["Last"]
            df2 = df2[df2["Last"] == temporary]

            return df2

        if (len(df) <= 0):
            return df

    def callinvest(self, symbol, entryD, exitD, percentageBuy, percentageSell):
        entryD = pd.to_datetime(np.datetime64(entryD))
        exitD = pd.to_datetime(np.datetime64(exitD))
        date = entryD
        callsCapital = []
        self.optionsdata["ID"] = np.arange(len(self.optionsdata))
        finalD = exitD
        while (date <= finalD):
            #print("-------------------DATE--------------" + str(date))
            #print(self.boughtCalls)
            #print(self.soldCalls)
            temp = self.stockdata[(self.stockdata.Date == date) & (self.stockdata.UnderlyingSymbol == symbol)]
            if (temp["UnderlyingSymbol"].values.size > 0):
                for i in self.boughtCalls:
                    if (i.values.size > 0):
                        if((i.Expiration.values == date) & (i.UnderlyingSymbol.values == symbol)):
                            if(i.Strike.values < temp.Close.values):
                                for k in range(len(callsCapital)):
                                    for s in range(len(callsCapital[k])):
                                        if (callsCapital[k][s][0] == i.ID.values):
                                            tempo = callsCapital[k][s][1]
                                            callsCapital[k][s][1] = tempo + (temp.Close.values - i.Strike.values)*100
                                            callsCapital[k][2] += (temp.Close.values - i.Strike.values)*100
                                self.capital += (temp.Close.values - i.Strike.values)*100
                               # self.boughtCalls.remove(self.boughtCalls[self.boughtCalls["ID"].values == i.ID.values])
                               # print(i)
                                #print(self.boughtCalls[0])
                                remove_from_array(self.boughtCalls, i.ID.values)
                                #print("Now selling "  + str(i.UnderlyingSymbol.values) + " with an expiration date " + str(i.Expiration.values) + " and a strike price of " + str(i.Strike.values) +" for a premium of " + str(i.Last.values))
                            #else:
                                #print("We let the option for the underlying " +  str(i.UnderlyingSymbol.values) + " expire.")

                for i in self.soldCalls:
                    if (i.values.size > 0):
                        if((i.Expiration.values == date) & (i.UnderlyingSymbol.values == symbol)):
                            if(i.Strike.values < temp.Close.values):
                                for k in range(len(callsCapital)):
                                    for s in range(len(callsCapital[k])):
                                        if (callsCapital[k][s][0] == i.ID.values):
                                            tempo = callsCapital[k][s][1]
                                            callsCapital[k][s][1] = tempo - (temp.Close.values - i.Strike.values)*100
                                            callsCapital[k][2] -= (temp.Close.values - i.Strike.values)*100
                                self.capital += (i.Strike.values - temp.Close.values)*100
                                self.totalInvestedCapitalcapital +=  temp.Close.values*100
                                #self.soldCalls.remove(i.values)
                                remove_from_array(self.soldCalls, i.ID.values)
                                #print("Now selling "  + str(i.UnderlyingSymbol.values) + " with an expiration date " + str(i.Expiration.values) + " and a strike price of " + str(i.Strike.values) +" for a premium of " + str(i.Last.values) + " to the Call Holder")
                            #else:
                                #print("The Call holder lets the option for the underlying " +  str(i.UnderlyingSymbol.values) + " expire.")

                if ((date >= entryD) & (temp["UnderlyingSymbol"].values == symbol)):
                    if ((temp["TradingPoints"].values == "0") | (temp["TradingPoints"].values == "2") ):
                        if (date < exitD):
                            #sellCall = self.buyCalls(self.optionsdata, temp.Close.values, percentageSell, date, symbol, 60, False, False, entryD)
                            buyCall =self.buyCalls(self.optionsdata, temp.Close.values, percentageBuy, date, symbol, 60, False, True, entryD)
                            sellCall =self.sellCalls(self.optionsdata, date, symbol, 60, entryD, buyCall.Strike.values, False)
                            if ((len(sellCall) > 0) & (len(buyCall) > 0)):
                                self.soldCalls.append(sellCall)
                                self.boughtCalls.append(buyCall)
                                transTemp1 = [[buyCall.ID.values, (-1)*buyCall.Last.values*100],[sellCall.ID.values, sellCall.Last.values*100], sellCall.Last.values*100 - buyCall.Last.values*100]
                                callsCapital.append(transTemp1)
                                self.capital = self.capital - buyCall.Last.values*100
                                self.totalInvestedCapital = self.totalInvestedCapital + buyCall.Last.values*100
                                self.capital = self.capital + sellCall.Last.values*100
                                if(buyCall.Expiration.values > finalD):
                                    finalD = buyCall.Expiration.values

                            #print( str(sellCall.ID.values) + "Now selling "  + str(sellCall.UnderlyingSymbol.values) + " with an expiration date " + str(sellCall.Expiration.values) + " and a strike price of " + str(sellCall.Strike.values) +" for a premium of " + str(sellCall.Last.values))
                            #print("Now buying "  + str(buyCall.UnderlyingSymbol.values) + " with an expiration date " + str(buyCall.Expiration.values) + " and a strike price of " + str(buyCall.Strike.values) +" for a premium of " + str(buyCall.Last.values))

                    #if (temp["TradingPoints"].values == "5"):
                     #   for i in self.boughtCalls:
                      #      if (i.values.size > 0):
                       #         if ((i.UnderlyingSymbol.values == symbol) & (i.Strike.values < temp.Close.values)):
                        #            for k in range(len(callsCapital)):
                         #               for s in range(len(callsCapital[k])):
                          #                  if (callsCapital[k][s][0] == i.ID.values):
                           #                     tempo = callsCapital[k][s][1]
                            #                    callsCapital[k][s][1] = tempo + (temp.Close.values - i.Strike.values)*100
                             #                   callsCapital[k][2] += (temp.Close.values - i.Strike.values)*100
                              #      self.capital += temp.Close.values - i.Strike.values
                               #     #self.boughtCalls.remove(i)
                                #    remove_from_array(self.boughtCalls, i.ID.values)
                                    #print("Now selling "  + str(i.UnderlyingSymbol.values) + " with an expiration date " + str(i.Expiration.values) + " and a strike price of " + str(i.Strike.values) +" for a premium of " + str(i.Last.values))

            date += timedelta(days=1)
        #print(callsCapital)
        alpha = 0
        for f in range(len(callsCapital)):
            alpha += callsCapital[f][2]


        return alpha


In [ ]:
tester = investor(1, 1000000, symbols, optionsdrop)
x = tester.stockinvestNew({"MCO": 100, "NEE":100}, "", "2018-05-05", "2018-07-05", True, 0.0)
print(x)

In [ ]:
tester2 = investor(1, 10000, symbols, optionsdrop)
tester2.buy({"MCO": 100, "NEE":100}, "", "2018-05-05", "2018-07-05", True)

In [ ]:
investorstock = investor(1, 10000, symbols, optionsdrop)

In [ ]:
investorstock.stockinvest(100, "NEE","" , "2018-05-05", "2018-07-05", True)

In [ ]:
investor2 = investor(1, 100000, symbols, optionsdrop)
investor2.callinvest("NEE", "2018-05-05", "2018-07-05", 0.05, 0.1)


In [ ]:
optionsdrop = optionsdrop.sort_values(by = "DataDate")
optionsdrop[(optionsdrop["UnderlyingSymbol"] == "NEE") & (optionsdrop["DataDate"] >= pd.to_datetime(np.datetime64("2018-05-29"))) & (optionsdrop["DataDate"] <= np.datetime64("2018-05-30"))]

In [ ]:
import os
#import glob
#os.chdir("/Users/Bjorn/Desktop/Master/Schoder Seminar/Paper/Datenanalyse/Data/Options/Collection")

#extension = 'csv'
#all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
#combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
#combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [ ]:
#options = pd.read_csv("/Users/Bjorn/Desktop/Master/Schoder Seminar/Paper/Datenanalyse/Data/combined_csv.csv")

In [ ]:
#options_reduced = combined_csv[(combined_csv["UnderlyingSymbol"] == "MCO") | (combined_csv["UnderlyingSymbol"] == "TXN") | (combined_csv["UnderlyingSymbol"] == "NEE") | (combined_csv["UnderlyingSymbol"] == "TFC") | (combined_csv["UnderlyingSymbol"] == "RF")]

In [ ]:
#options_reduced.to_csv("options_reduced.csv", index=False, encoding='utf-8-sig')

In [ ]:
#options_reduced.head()

In [ ]:
options.head()

In [ ]:
symbols[symbols.Date == pd.to_datetime(np.datetime64("2018-05-10"))]

In [ ]:
tester = investor(1, 10000, symbols, options)
tester.buy({"MCO": 50, "TXN":100}, "sellAll", "2018-05-10", "2018-07-05")
#print(tester.assetValue)
#print(tester.capital)
#tester.getAssetList()

In [ ]:
tester2 = investor(1, 1000, symbols, options)
tester2.compete([[{"MCO": 2, "NEE": 2, "TFC": 5}, "", "2018-05-02", "2018-06-20"], [{"NEE": 2}, "", "2018-05-10", "2018-05-20"]])

In [ ]:
investor2 = investor(1, 100000000, symbols, optionsdrop)
investor2.callinvest("NEE","2018-05-10", "2018-07-07", 0.05, 0.1)
#print(investor2.capital)

In [ ]:
optionsdrop[optionsdrop["UnderlyingSymbol"] == "TFC"]

In [103]:
tester = investor(1, 10000, symbols, options)
tester.buy({"MCO": 50, "TXN":100}, "sellAll", "2018-05-10", "2018-07-05")
#print(tester.assetValue)
#print(tester.capital)
#tester.getAssetList()

Now sold all Shares of Share MCO
Now sold all Shares of Share MCO
Now sold all Shares of Share MCO
Now sold all Shares of Share MCO
2018-07-05 00:00:00
The profit that was generated by buying and selling stocks of [-1.4308097]
Now sold all Shares of Share TXN
Now sold all Shares of Share TXN
2018-07-05 00:00:00
The profit that was generated by buying and selling stocks of [8.42817175]


In [169]:
tester2 = investor(1, 1000, symbols, options)
tester2.compete([[{"MCO": 2, "NEE": 2, "TFC": 5}, "", "2018-05-02", "2018-06-20"], [{"NEE": 2}, "", "2018-05-10", "2018-05-20"]])

Now buying 2 Shares of Share MCO
The worker owns 2 of Asset MCO
Now selling 1 Shares of Share MCO
The worker owns 1 of Asset MCO
Now sold all Shares of Share MCO
The worker owns 0 of Asset MCO
Now buying 2 Shares of Share MCO
The worker owns 2 of Asset MCO
Now selling 1 Shares of Share MCO
The worker owns 1 of Asset MCO
Now sold all Shares of Share MCO
The worker owns 0 of Asset MCO
Now buying 2 Shares of Share MCO
The worker owns 2 of Asset MCO
Now selling 1 Shares of Share MCO
The worker owns 1 of Asset MCO
The profit that was generated by buying and selling stocks of MCO amounts to [-189.42]
Now buying 2 Shares of Share NEE
The worker owns 1 of Asset MCO
The worker owns 2 of Asset NEE
Now selling 1 Shares of Share NEE
The worker owns 1 of Asset MCO
The worker owns 1 of Asset NEE
Now sold all Shares of Share NEE
The worker owns 1 of Asset MCO
The worker owns 0 of Asset NEE
Now buying 2 Shares of Share NEE
The worker owns 1 of Asset MCO
The worker owns 2 of Asset NEE
Now selling 1 Sha

,UnderlyingSymbol,UnderlyingPrice,Exchange,OptionRoot,OptionExt,Type,Expiration,DataDate,Strike,Last,Bid,Ask,Volume,OpenInterest,T1OpenInterest
1600,MCO,172.52,*,MCO180615C00185000,NaN,call,2018-06-15,2018-05-18,185.0,0.39,0.05,0.22,0,5,5
3078,MCO,173.42,*,MCO180615C00185000,NaN,call,2018-06-15,2018-05-30,185.0,0.15,0.06,0.13,0,11,11
4552,MCO,173.95,*,MCO180615C00185000,NaN,call,2018-06-15,2018-05-24,185.0,0.25,0.12,0.22,0,6,6
6082,MCO,174.03,*,MCO180615C00185000,NaN,call,2018-06-15,2018-05-25,185.0,0.15,0.10,0.18,5,6,0
7612,MCO,170.57,*,MCO180615C00185000,NaN,call,2018-06-15,2018-05-31,185.0,0.15,0.00,0.22,0,11,0
20952,MCO,174.36,*,MCO180615C00185000,NaN,call,2018-06-15,2018-05-22,185.0,0.25,0.22,0.28,2,5,6
22412,MCO,174.05,*,MCO180615C00185000,NaN,call,2018-06-15,2018-05-23,185.0,0.25,0.15,0.25,0,6,6
26800,MCO,175.09,*,MCO180615C00185000,NaN,call,2018-06-15,2018-05-21,185.0,0.39,0.24,0.37,0,5,5
28310,MCO,170.55,*,MCO180615C00185000,NaN,call,2018-06-15,2018-05-09,185.0,0.25,0.18,0.26,5,1,5
29800,MCO,167.89,*,MCO180615C00185000,NaN,call,2018-06-15,2018-05-08,185.0,0.51,0.10,0.20,0,1,1


In [17]:
investor2 = investor(1, 100000000, symbols, optionsdrop)
investor2.callinvest("NEE","2018-05-10", "2018-07-07", 0.05, 0.1)
#print(investor2.capital)

13
11
7


array([-145.])

In [128]:
optionsdrop[optionsdrop["UnderlyingSymbol"] == "TFC"]

,UnderlyingSymbol,UnderlyingPrice,Exchange,OptionRoot,OptionExt,Type,Expiration,DataDate,Strike,Last,Bid,Ask,Volume,OpenInterest,T1OpenInterest,ID
